In [66]:
!pip install inflect --upgrade
import re, string, unicodedata
import nltk
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

Requirement already up-to-date: inflect in /usr/local/lib/python3.6/dist-packages (2.1.0)


# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [9]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "
print(whitespace_string)




  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [33]:
 ##### Your Code Here #####
import re
whitespace_string = whitespace_string.replace('\n', ' ') # Use to remove carriage returns. 
whitespace_string = re.sub(' +', ' ', whitespace_string) # Remove excess whitespace. 
print(whitespace_string)

 This is a string that has a lot of extra whitespace. 


### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [32]:
##### Your Code Here #####
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt", sep = "(?:,|\s)\s*", header = None, engine='python')
df = df.rename(columns={0: "Month", 1: "Day", 2: "Year"})
df = df[["Day","Month","Year"]]
df.head()

,Day,Month,Year
0,8,March,2015
1,15,March,2015
2,22,March,2015
3,29,March,2015
4,5,April,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [68]:
df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv")
df.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [69]:
import unicodedata, inflect

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words or string"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words or string, also 
    lowercases every word.
    """
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        new_word = word.lower()
        if new_word != '':
            new_words.append(new_word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    #words = remove_stopwords(words)
    return words
  
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words or string 
    with textual representation.
    """
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

#def remove_stopwords(words):
#    """Remove stop words from list of tokenized words"""
#    new_words = []
#    for word in words:
#        if word not in stopwords:
#            new_words.append(word)
#    return new_words
  
  
tokenizer = RegexpTokenizer(r'\w+')
df['SentimentText'] = normalize(df['SentimentText'])
df['SentimentText'] = df['SentimentText'].apply(tokenizer.tokenize)
df['SentimentText'] = [replace_numbers(token) for token in df['SentimentText']]
df.head()

,Sentiment,SentimentText
0,0,"[is, so, sad, for, my, apl, friend]"
1,0,"[i, missed, the, new, moon, trailer]"
2,1,"[omg, its, already, seven, thirty, o]"
3,0,"[omgaga, im, sooo, im, gunna, cry, i, ve, been..."
4,0,"[i, think, mi, bf, is, cheating, on, me, t_t]"


In [51]:
df.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


### How should TF-IDF scores be interpreted? How are they calculated?

#### Your Answer Here #####

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



In [0]:
##### Your Code Here #####

# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [0]:
##### Your Code Here #####